In [ ]:
pip install torch torchvision gensim

In [ ]:
!python -m spacy download el_core_news_sm

In [ ]:
# !pip install --upgrade pandas

In [ ]:
import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.feature_extraction.text import CountVectorizer
import nltk
from nltk.tokenize import word_tokenize
from unidecode import unidecode
import re
import spacy
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem import PorterStemmer
spacy.load('el_core_news_sm')
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords
nlp = spacy.load("el_core_news_sm")

# Import Data

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

df = pd.read_csv("/kaggle/input/ys19-2023-assignment-2/train_set.csv")
test_df = pd.read_csv("/kaggle/input/ys19-2023-assignment-2/test_set.csv")
valid_df = pd.read_csv("/kaggle/input/ys19-2023-assignment-2/valid_set.csv")

df['Label_Map'] = df['Sentiment'].map({
    'NEGATIVE' : 0, 'NEUTRAL' : 1, 'POSITIVE' : 2
})
valid_df['Label_Map'] = valid_df['Sentiment'].map({
    'NEGATIVE' : 0, 'NEUTRAL' : 1, 'POSITIVE' : 2
})

# Preprocessing functions

In [ ]:
def lowercase_without_accentds(text):
    chars = "ΆΈΊΌΎΉάέίόύήώ"
    lchars = "αειουηαειουηω"
    return text.translate(str.maketrans(chars,lchars))

def extract_greek_characters(input_string):
    greek_pattern = re.compile('[Α-Ωα-ω]+')
    greek_characters = greek_pattern.findall(input_string)
    result = ' '.join(greek_characters)
    return result

greek_stopwords = ["ο","ειναι","απο","https","co","λεπω","λεμε","ειπα","ειπες","εβαλα","αδιακοπα","αι","ακομα","ακομη","ακριβως","αληθεια","αληθινα","αλλα","αλλαχου","αλλες","αλλη","αλλην","αλλης","αλλιως","αλλιωτικα","αλλο","αλλοι","αλλοιως","αλλοιωτικα","αλλον","αλλος","αλλοτε","αλλου","αλλους","αλλων","αμα","αμεσα","αμεσως","αν","ανα","αναμεσα","αναμεταξυ","ανευ","αντι","αντιπερα","αντις","ανω","ανωτερω","αξαφνα","απ","απεναντι","απο","αποψε","αρα","αραγε","αργα","αργοτερο","αριστερα","αρκετα","αρχικα","ας","αυριο","αυτα","αυτες","αυτη","αυτην","αυτης","αυτο","αυτοι","αυτον","αυτος","αυτου","αυτους","αυτων","αφοτου","αφου","βεβαια","βεβαιοτατα","γι","για","γρηγορα","γυρω","δα","δε","δεινα","δεν","δεξια","δηθεν","δηλαδη","δι","δια","διαρκως","δικα","δικο","δικοι","δικος","δικου","δικους","διολου","διπλα","διχως","εαν","εαυτο","εαυτον","εαυτου","εαυτους","εαυτων","εγκαιρα","εγκαιρως","εγω","εδω","ειδεμη","ειθε","ειμαι","ειμαστε","ειναι","εις","εισαι","εισαστε","ειστε","ειτε","ειχα","ειχαμε","ειχαν","ειχατε","ειχε","ειχες","εκαστα","εκαστες","εκαστη","εκαστην","εκαστης","εκαστο","εκαστοι","εκαστον","εκαστος","εκαστου","εκαστους","εκαστων","εκει","εκεινα","εκεινες","εκεινη","εκεινην","εκεινης","εκεινο","εκεινοι","εκεινον","εκεινος","εκεινου","εκεινους","εκεινων","εκτος","εμας","εμεις","εμενα","εμπρος","εν","ενα","εναν","ενας","ενος","εντελως","εντος","εντωμεταξυ","ενω","εξ","εξαφνα","εξης","εξισου","εξω","επανω","επειδη","επειτα","επι","επισης","επομενως","εσας","εσεις","εσενα","εστω","εσυ","ετερα","ετεραι","ετερας","ετερες","ετερη","ετερης","ετερο","ετεροι","ετερον","ετερος","ετερου","ετερους","ετερων","ετουτα","ετουτες","ετουτη","ετουτην","ετουτης","ετουτο","ετουτοι","ετουτον","ετουτος","ετουτου","ετουτους","ετουτων","ετσι","ευγε","ευθυς","ευτυχως","εφεξης","εχει","εχεις","εχετε","εχθες","εχομε","εχουμε","εχουν","εχτες","εχω","εως","η","ηδη","ημασταν","ημαστε","ημουν","ησασταν","ησαστε","ησουν","ηταν","ητανε","ητοι","ηττον","θα","ι","ιδια","ιδιαν","ιδιας","ιδιες","ιδιο","ιδιοι","ιδιον","ιδιος","ιδιου","ιδιους","ιδιων","ιδιως","ιι","ιιι","ισαμε","ισια","ισως","καθε","καθεμια","καθεμιας","καθενα","καθενας","καθενος","καθετι","καθολου","καθως","και","κακα","κακως","καλα","καλως","καμια","καμιαν","καμιας","καμποσα","καμποσες","καμποση","καμποσην","καμποσης","καμποσο","καμποσοι","καμποσον","καμποσος","καμποσου","καμποσους","καμποσων","κανεις","κανεν","κανενα","κανεναν","κανενας","κανενος","καποια","καποιαν","καποιας","καποιες","καποιο","καποιοι","καποιον","καποιος","καποιου","καποιους","καποιων","καποτε","καπου","καπως","κατ","κατα","κατι","κατιτι","κατοπιν","κατω","κιολας","κλπ","κοντα","κτλ","κυριως","λιγακι","λιγο","λιγωτερο","λογω","λοιπα","λοιπον","μα","μαζι","μακαρι","μακρυα","μαλιστα","μαλλον","μας","με","μεθαυριο","μειον","μελει","μελλεται","μεμιας","μεν","μερικα","μερικες","μερικοι","μερικους","μερικων","μεσα","μετ","μετα","μεταξυ","μεχρι","μη","μηδε","μην","μηπως","μητε","μια","μιαν","μιας","μολις","μολονοτι","μοναχα","μονες","μονη","μονην","μονης","μονο","μονοι","μονομιας","μονος","μονου","μονους","μονων","μου","μπορει","μπορουν","μπραβο","μπρος","να","ναι","νωρις","ξανα","ξαφνικα","ο","οι","ολα","ολες","ολη","ολην","ολης","ολο","ολογυρα","ολοι","ολον","ολονεν","ολος","ολοτελα","ολου","ολους","ολων","ολως","ολωςδιολου","ομως","οποια","οποιαδηποτε","οποιαν","οποιανδηποτε","οποιας","οποιαςδηποτε","οποιδηποτε","οποιες","οποιεςδηποτε","οποιο","οποιοδηποτε","οποιοι","οποιον","οποιονδηποτε","οποιος","οποιοςδηποτε","οποιου","οποιουδηποτε","οποιους","οποιουςδηποτε","οποιων","οποιωνδηποτε","οποτε","οποτεδηποτε","οπου","οπουδηποτε","οπως","ορισμενα","ορισμενες","ορισμενων","ορισμενως","οσα","οσαδηποτε","οσες","οσεςδηποτε","οση","οσηδηποτε","οσην","οσηνδηποτε","οσης","οσηςδηποτε","οσο","οσοδηποτε","οσοι","οσοιδηποτε","οσον","οσονδηποτε","οσος","οσοςδηποτε","οσου","οσουδηποτε","οσους","οσουςδηποτε","οσων","οσωνδηποτε","οταν","οτι","οτιδηποτε","οτου","ου","ουδε","ουτε","οχι","παλι","παντοτε","παντου","παντως","παρα","περα","περι","περιπου","περισσοτερο","περσι","περυσι","πια","πιθανον","πιο","πισω","πλαι","πλεον","πλην","ποια","ποιαν","ποιας","ποιες","ποιο","ποιοι","ποιον","ποιος","ποιου","ποιους","ποιων","πολυ","ποσες","ποση","ποσην","ποσης","ποσοι","ποσος","ποσους","ποτε","που","πουθε","πουθενα","πρεπει","πριν","προ","προκειμενου","προκειται","προπερσι","προς","προτου","προχθες","προχτες","πρωτυτερα","πως","σαν","σας","σε","σεις","σημερα","σιγα","σου","στα","στη","στην","στης","στις","στο","στον","στου","στους","στων","συγχρονως","συν","συναμα","συνεπως","συνηθως","συχνα","συχνας","συχνες","συχνη","συχνην","συχνης","συχνο","συχνοι","συχνον","συχνος","συχνου","συχνου","συχνους","συχνων","συχνως","σχεδον","σωστα","τα","ταδε","ταυτα","ταυτες","ταυτη","ταυτην","ταυτης","ταυτο,ταυτον","ταυτος","ταυτου","ταυτων","ταχα","ταχατε","τελικα","τελικως","τες","τετοια","τετοιαν","τετοιας","τετοιες","τετοιο","τετοιοι","τετοιον","τετοιος","τετοιου","τετοιους","τετοιων","τη","την","της","τι","τιποτα","τιποτε","τις","το","τοι","τον","τος","τοσα","τοσες","τοση","τοσην","τοσης","τοσο","τοσοι","τοσον","τοσος","τοσου","τοσους","τοσων","τοτε","του","τουλαχιστο","τουλαχιστον","τους","τουτα","τουτες","τουτη","τουτην","τουτης","τουτο","τουτοι","τουτοις","τουτον","τουτος","τουτου","τουτους","τουτων","τυχον","των","τωρα","υπ","υπερ","υπο","υποψη","υποψιν","υστερα","φετος","χαμηλα","χθες","χτες","χωρις","χωριστα","ψηλα","ω","ωραια","ως","ωσαν","ωσοτου","ωσπου","ωστε","ωστοσο","ωχ","ο","η","το","τα","τη","δηλαδη","μεχρι","γιατι","εχω","στους","μια","ένας","μία","κάποιος","κάποια","κάποιο","κάποιοι","αυτος","αυτη","αυτο","αυτοι","αυτες","αυτα","στο","στη","στα","για","με","απο","προς","ειναι","εχει","εχουν","θα","δεν","πανω","κατω","μεσα","εξω","κατω","ως","πανω","κατω","πιο","εδω","εκει","πολυ","λιγο","τωρα","ακομα","ομως","επισης","παντα","ακομη","πιθανως","μονο","οχι","ναι","ευχαριστως","γενικα","ολοι","ολες","ολα","ποιος","ποια","ποιο","ποιοι","ποιες","τιποτα","κανεις","καμια","κανενα","κανενες","αυτος","αυτη","αυτο","αυτοι","αυτες","αυτα","απο","σε","υπο","μετα","πριν","επειτα","αντι","εναν","μιαν","κανεναν","καμιαν","κανενα","καμια","μιαν","ενα","οποιος","οποια","οποιο","οποιοι","οποιες","οποιαν","οποιον","ολος","ολη","ολα","ολους","ολες","ολων","καθενας","καθεμια","καθενα","καθενες","ακομα","ενω","επομενως","συνεπως","επιπλεον","παρολα αυτα","παρ ολα αυτα","επισης","και","αλλα","αλλα και","αν","εαν","αν και","αντι","αντι να","αντι το","αντι τα","αντι του","αντι τη","αντι των","αντι στο","αντι στη","αντι στα","αντι στου","αντι στην","αντι στις","αντι στον","μεσω","τους","μας","ηταν","εκ","φορα","πρωτη","ειχα","εμεις","εσεις","ηδη","απ","εγινε","ειχε","αλλα","ουτε","ενας","εσας","αυτοι","αυτο","νεα","οντως","θελετε","κανει","σ","μας","πρεπε","ε","μαλιστα","τους","ηθελε","παω","εβαλε","λεει","γ","ν","θες","ερχεται","διαρκεια","θελουν","ασε","χ","λες","ξερω","α","δω","ειδε","μπηκε","βαλει","μερες","εφοσον","ενα","δυο","τρια","γινει","εργο","μιλαω","μιλησε","ποσα","ωρες","πρωινες","πρωτα","θελει","βαζω","εβαζε","εναντι","μπορεις","βρισκει","δει","μπορω","γινε","κανουν"]
pattern = '#!$%^&*()_+={}\[\]:;"\'<>,.?/\\|`~-><]'

def process_text(text):
    
#     new_text = extract_greek_characters(text)
    doc = nlp(text)
    processed_words = []
    # Loop through each word
    for word in doc:
        if word.is_punct or any(char.isdigit() for char in word.text) or word.text in greek_stopwords:
            continue
        else:
            clean_word = word.lemma_
            clean_word = lowercase_without_accentds(clean_word)
            clean_word = clean_word.lower()
            if clean_word not in greek_stopwords:
                processed_words.append(clean_word)
    # Join the processed words back into a string
    processed_text = ' '.join(processed_words)
    return processed_text

# Data preprocessing

In [ ]:
import os
# os.remove('train.json')
# os.remove('valid.json')
text = df['Text']
valid_text = valid_df['Text']
test_text = test_df['Text']
print("Preproccesing training data")
df['processed_text'] = text.apply(process_text)
df.to_json('train.json', orient='records', lines=True)
print("Preproccesing valid data")
valid_df['processed_text'] = valid_text.apply(process_text)
valid_df.to_json('valid.json', orient='records', lines=True)
test_df['processed_text'] = test_text.apply(process_text)
test_df.to_json('test.json', orient='records', lines=True)
print("Vectorization")

# Get preprocessed data from file

In [ ]:
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
# Write the DataFrame to a JSON file
df_read = pd.read_json('train.json', lines=True)
train_preprocessed_text = df_read['processed_text']
valid_df_read = pd.read_json('valid.json', lines=True)
valid_preprocessed_text = valid_df_read['processed_text']
test_df_read = pd.read_json('test.json', lines=True)
test_preprocessed_text = test_df_read['processed_text']

print("TRAIN")
print(len(train_preprocessed_text))
print("VALID")
print(len(valid_preprocessed_text))
print("TEST")
print(len(test_preprocessed_text))

# Data tokenization

In [ ]:
import re
from nltk.tokenize import word_tokenize

def clean_and_tokenize(text):

    pattern = r'[!@#$%^&*()_+={}\[\]:;<>,.?/~`]+'
    
    cleaned_text = re.sub(pattern, '', text)

    tokens = cleaned_text.split()
    
    return tokens

# Tokenize model

In [ ]:
from gensim.models import Word2Vec
import nltk
nltk.download('punkt')

# Preprocess each sentence
df_read['Tokenized_text'] = train_preprocessed_text.apply(clean_and_tokenize)
valid_df_read['Tokenized_text'] = valid_preprocessed_text.apply(clean_and_tokenize)
test_df_read['Tokenized_text'] = test_preprocessed_text.apply(clean_and_tokenize)

# Visualize model

In [ ]:
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
from gensim.models import Word2Vec
import pandas as pd
import numpy as np

# Assuming df_read and valid_df_read are your dataframes with 'Tokenized_text' column

# Combine the tokenized text from both dataframes
all_tokenized_text = pd.concat([df_read['Tokenized_text'], valid_df_read['Tokenized_text'],test_df_read['Tokenized_text']])

# Define the vector_size (you need to set it to the appropriate value)
vector_size = 100

# Visualized words
visualized_words = ['μητσοτακης', 'τσιπρας', 'νδ', 'συριζα', 'κουλη', 'κκε', 'πολιτης', 'τσιπρα', 'βουλευτης', 'κυριακος',
                   'προθυπουργος' , 'πασοκ' , 'πολιτικος' , 'δημοκρατια' , 'νεος' , 'βελοπουλος']

def visualize_embeddings(embeddings, words, axs, title):
    tsne = TSNE(init = "pca",n_components=2,random_state=42, perplexity=len(words) - 1)
    embedding_vectors = np.array([embeddings[word] for word in words])
    two_d_embeddings = tsne.fit_transform(embedding_vectors)

    for i, word in enumerate(words):
        x, y = two_d_embeddings[i, :]
        axs.scatter(x, y)
        axs.annotate(word, (x, y), xytext=(5, 2), textcoords="offset points", ha="right", va="bottom")

    axs.set_title(title)

# Create subplots
fig, axs = plt.subplots(2, 3, figsize=(15, 10))

# Iterate over the subplots
count = 2
for i in range(2):
    for j in range(3):
        axs_ = axs[i, j]

        # Train Word2Vec model with CBOW algorithm (sg=0)
        w2v_model = Word2Vec(sentences=all_tokenized_text, vector_size=vector_size, window = count, min_count=1, sg=0)
        
        # Visualize embeddings
        visualize_embeddings(w2v_model.wv, visualized_words, axs_, f'CBOW, Window Size: {count}')
        count += 3
# Show the plots
plt.show()

# Create new subplots for the second set of visualizations
fig, axs = plt.subplots(2, 3, figsize=(15, 10))
count = 2
for i in range(2):
    for j in range(3):
        axs_ = axs[i, j]

        # Train Word2Vec model with Skip-gram algorithm (sg=1)
        w2v_model = Word2Vec(sentences=all_tokenized_text, vector_size=vector_size, window=count, min_count=1, sg=1)

        # Visualize embeddings
        visualize_embeddings(w2v_model.wv, visualized_words, axs_, f'Skip-gram, Window Size: {count}')
        count += 3

# Show the plots
plt.show()

# Train word2vec
I choose cbow algorithmn and window size = 8

In [ ]:
vector_size = 100
w2v_model = Word2Vec(sentences=pd.concat([df_read['Tokenized_text'], valid_df_read['Tokenized_text'],test_df_read['Tokenized_text']]), vector_size=vector_size, window=8, min_count=1, sg=0)
w2v_model.save("greek_word2vec.model")

# Create training torches

In [ ]:
import torch
from torch.nn.utils.rnn import pad_sequence
import numpy as np
from gensim.models import Word2Vec
from statistics import mean

# Load the Word2Vec model
w2v_model = Word2Vec.load("greek_word2vec.model")

# Function to get word embeddings for a tokenized tweet
def get_word_embeddings(tokenized_tweet, model):
    embeddings = []
    if not tokenized_tweet:
        tokenized_tweet = ["Ολυμπιακός"] # word without sence
    for word in tokenized_tweet:        
        if word in model.wv:
            embeddings.append(model.wv[word])
        else:
            embeddings.append(np.zeros(model.vector_size, dtype=np.float32))
    return embeddings

# Function to get mean embeddings for a list of tweets
def get_embeddings_by_mean(tweets):
    return np.mean(np.stack(tweets, axis=0), axis=0)


word_embeddings = [get_word_embeddings(tokenized_tweet, w2v_model) for tokenized_tweet in df_read['Tokenized_text']]
word_embeddings_valid = [get_word_embeddings(tokenized_tweet, w2v_model) for tokenized_tweet in valid_df_read['Tokenized_text']]
word_embeddings_test = [get_word_embeddings(tokenized_tweet, w2v_model) for tokenized_tweet in test_df_read['Tokenized_text']]

# Convert to tensors
X_train = torch.tensor([get_embeddings_by_mean(tweet) for tweet in word_embeddings])
X_valid = torch.tensor([get_embeddings_by_mean(tweet) for tweet in word_embeddings_valid])
X_test = torch.tensor([get_embeddings_by_mean(tweet) for tweet in word_embeddings_test])

# Convert labels to tensors
Y_train = torch.tensor(df['Label_Map'])
Y_valid = torch.tensor(valid_df['Label_Map'])


print(X_train.shape)
print(Y_train.shape)
print(X_valid.shape)
print(Y_valid.shape)
print(X_test.shape)

# Dataset class

In [ ]:
# Define the dataset class
class TextDataset:
    def __init__(self, features, targets):
        self.features = features
        self.targets = targets

    def __len__(self):
        return self.features.shape[0]

    def __getitem__(self, item):
        return {
            "x": self.features[item].clone().detach().to(torch.float),
            "y": self.targets[item].clone().detach().to(torch.long)
        }

# Engine class

In [ ]:
import torch
import torch.nn as nn 
from sklearn.metrics import f1_score

import torch.nn.functional as F

class Engine:
    def __init__(self, model, optimizer):
        self.model = model
        self.optimizer = optimizer
        
    
    @staticmethod
    def loss_fn(targets, outputs):
        return nn.CrossEntropyLoss()(outputs, targets)
    
    
    def train(self,data_loader):
        self.model.train()
        final_loss = 0
        for data in data_loader:
            self.optimizer.zero_grad()
            inputs = data["x"]
            targets = data["y"]
            outputs = self.model(inputs)
            loss = self.loss_fn(targets,outputs)
            loss.backward()
            self.optimizer.step()
            final_loss += loss.item()
        return final_loss / len(data_loader)
    
    def evaluate(self,data_loader):
        self.model.eval()
        final_loss = 0
        all_targets = []
        all_outputs = []
        all_probabilities = []
        for data in data_loader:
            inputs = data["x"]
            targets = data["y"]
            outputs = self.model(inputs)
            
            # Apply softmax to obtain probabilities
            probabilities = F.softmax(outputs, dim=1)
            all_probabilities.extend(probabilities.cpu().detach().numpy())
            
            loss = self.loss_fn(targets,outputs)
            final_loss += loss.item()
            
            _, predictions = torch.max(outputs, 1)
            all_targets.extend(targets.cpu().numpy())
            all_outputs.extend(predictions.cpu().numpy())
        f1_per_class = f1_score(all_targets, all_outputs, average=None)
        f1 = f1_score(all_targets, all_outputs, average='micro')
        return final_loss / len(data_loader), [f1,f1_per_class],all_probabilities,all_outputs
    
    def make_predictions(self,model, data_loader):
        self.model.eval()
        all_probabilities = []
        all_outputs = []

        with torch.no_grad():
            for data in data_loader:
                inputs = data["x"]
                outputs = self.model(inputs)

                # Apply softmax to obtain probabilities
                probabilities = F.softmax(outputs, dim=1)
                all_probabilities.extend(probabilities.cpu().detach().numpy())

                _, predictions = torch.max(outputs, 1)
                all_outputs.extend(predictions.cpu().numpy())

        return np.array(all_probabilities), np.array(all_outputs)

# Build model

In [ ]:
class Model(nn.Module):
    
    def __init__(self, nfeatures, ntargets , nlayers, hidden_size, dropout):
        super().__init__()
        layers = []
        for i in range(nlayers):
            if len(layers) == 0:
                layers.append(nn.Linear(nfeatures,hidden_size[i]))
#                 layers.append(nn.BatchNorm1d(hidden_size))
#                 layers.append(nn.Dropout(dropout))
                layers.append(nn.ReLU())
            else:
                layers.append(nn.Linear(hidden_size[i-1],hidden_size[i]))
#                 layers.append(nn.BatchNorm1d(hidden_size))
#                 layers.append(nn.Dropout(dropout))
                layers.append(nn.ReLU())
        layers.append(nn.Linear(hidden_size[i],ntargets))
        self.model = nn.Sequential(*layers)
    
    def forward(self, x):
        return self.model(x)

# Roc curve function

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc
from sklearn.preprocessing import label_binarize

def plot_roc_curve(y_true, probabilities):
    # Assuming probabilities is a list of numpy arrays
    probabilities = np.vstack(probabilities)

    # Binarize the labels
    y_true_bin = label_binarize(y_true, classes=[0, 1, 2])

    num_classes = probabilities.shape[1]

    plt.figure(figsize=(8, 8))  # Adjust the figure size as needed

    for i in range(num_classes):
        fpr, tpr, _ = roc_curve(y_true_bin[:, i], probabilities[:, i])
        roc_auc = auc(fpr, tpr)

        plt.plot(fpr, tpr, lw=2, label=f'Class {i} (AUC = {roc_auc:.2f})')

    plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('ROC Curve')
    plt.legend(loc="lower right")
    plt.show()

# Learning curve function

In [ ]:
def plot_learning_curve(train_losses, valid_losses):
    plt.figure(figsize=(10, 6))
    epochs = range(1, len(train_losses) + 1)

    plt.plot(epochs, train_losses, label='Training Loss', marker='o')
    plt.plot(epochs, valid_losses, label='Validation Loss', marker='o')

    plt.title('Learning Curve')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.grid(True)
    plt.show()

# Comfusion matrix

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns

def plot_confusion_matrix(y_true, y_pred, classes):
    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=classes, yticklabels=classes)
    plt.title('Confusion Matrix')
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.show()

# Model

In [ ]:
from torch.optim.lr_scheduler import StepLR
from torch.utils.data import Dataset, DataLoader

def run_training(f_,params,flag,model_save_path):
    
    dataset = TextDataset(X_train, Y_train)
    train_loader = DataLoader(dataset, batch_size=params["batch_size1"], shuffle=True)

    validation_set = TextDataset(X_valid, Y_valid)
    valid_loader = DataLoader(validation_set, batch_size=params["batch_size2"], shuffle=False)
    
    model = Model(nfeatures = X_train.shape[1],ntargets = 3,
            nlayers = params["num_layers"], hidden_size = params["hidden_size"], dropout = params["dropout"])
    optimizer = torch.optim.Adam(model.parameters(),lr=params["learning_rate"])
    eng = Engine(model,optimizer)
    
    EPOCHS = 20
    best_loss = np.inf
    early_stopping_iter = 100
    early_stopping_counter = 0
    total_probabilities = []
    train_losses = []
    valid_losses = []
    f1_best = 0
    best_epoch = 0
    r_classes_f1 = [0,0,0]
    scheduler = StepLR(optimizer, step_size=1, gamma=0.01)
    for epoch in range(EPOCHS):
        train_loss = eng.train(train_loader)
        valid_loss,f_list,probabilities,all_outputs = eng.evaluate(valid_loader)
        f1 = f_list[0]
        f_per_class = f_list[1]
        train_losses.append(train_loss)
        valid_losses.append(valid_loss)

        if f1 > f1_best:
            f1_best = f1
            best_epoch = epoch
            r_classes_f1 = f_per_class
            torch.save(model.state_dict(), model_save_path)
        else:
            early_stopping_counter +=1
        if early_stopping_counter > early_stopping_iter:
            break
    if flag == 1:
        #plot roc curve
        plot_roc_curve(Y_valid, probabilities)
        plot_learning_curve(train_losses,valid_losses)
        plot_confusion_matrix(Y_valid, all_outputs, ["Negative, Neutral, Positive"])
    return best_loss,[f1_best,best_epoch,r_classes_f1],eng

# Optuna optimazer

In [ ]:
def objective(trial):
    params = {
        "num_layers" : 3,
        "hidden_size" : 
        { 0 : trial.suggest_int("h1",1,300),
          1 : trial.suggest_int("h2",1,300),
          2 : trial.suggest_int("h3",1,300)
        },
        
        "dropout" : trial.suggest_uniform("dropout",0.1,0.4),
        "learning_rate" : trial.suggest_uniform("learning_rate", 1e-6,1e-3),
        "batch_size1" : trial.suggest_int("batch_size1",30,200),
        "batch_size2" : trial.suggest_int("batch_size2",30,200)
    }
    
    print(params)
    
    temp_loss,details,eng_opt = run_training(0, params,0,"optuna_best_model.pth")
    
    print(f"best_f1: {details[0]}, best_epoch {details[1]}")
    for i, f1 in enumerate(details[2]):
        print(f"Class {i}: {f1}")
    return details[0]

# Run optuna

In [ ]:
import optuna

study = optuna.create_study(direction = "maximize")
study.optimize(objective, n_trials=1) # change number of trials

print("best trial:")
trial_ = study.best_trial

print(trial_.values)
print(trial_.params)

# Using optimal optuna parameters

In [ ]:
trial_.params["num_layers"] = 3
trial_.params["hidden_size"] = { 
        0 : trial_.params["h1"],
        1 : trial_.params["h2"],
        2 : trial_.params["h3"]
}
    
print(trial_.params)

scr,results,eng_opt = run_training(j,trial_.params,1,"best_model.pth")
print(results[0],results[1])

for i, f1 in enumerate(results[2]):
        print(f"Class {i}: {f1}")

# Using for me optimal parameters

In [ ]:
optim_params = {
        "num_layers" : 3,
        "hidden_size" : 
        { 0 : 174,
          1 : 34,
          2 : 153
        },
        
        "dropout" : 0.13,
        "learning_rate" : 0.0001755,
        "batch_size1" : 151,
        "batch_size2" : 89
    }
src, results,eng = run_training(j,optim_params,1,"best_model.pth")
print(results[0],results[1])

for i, f1 in enumerate(results[2]):
        print(f"Class {i}: {f1}")

# Create output file

In [ ]:
model = torch.load('best_model.pth')

dataset_test = TextDataset(X_test, Y_train)
test_loader = DataLoader(dataset_test, batch_size=1, shuffle=False)

output = eng.make_predictions(model,test_loader)

predictions = output[1]

In [ ]:
import csv

number_to_sentiment = {
    0: "NEGATIVE",
    1: "NEUTRAL",
    2: "POSITIVE",
}

array = [number_to_sentiment[value] for value in predictions]

with open("submission.csv", mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["Id", "Predicted"])

    for idx, word in enumerate(array, start=1):
        writer.writerow([idx, word])